# Compression in Python – A Tutorial


### What is Compression?

Compression is a method used to shrink the size of files in order to save storage space on a device.  When downloading content from the internet, we often encounter files with the extension .zip or .rar.  These are instances of files that have been compressed.  Upon opening, they create the 'original', accessible file.

   There are diverse and complex schools of thought on how to approach compression, ranging from information theory to statistics.  And as we experience the age of big data, research in this field is in an incessant pursuit for more efficient compression methdos.  In general, the algorithms that make compression possible rely on some rather intuitive mathematical rules.  Text compression essentially remove redundancy from a file by removing redundant repeating characters (or patterns of characters), and keeping track of these in a dictionary.  For image compression, the same idea applies except instead of characters it manipulates pixels and their colors.  

### Data Compression Modules from the Python Standard Library (https://docs.python.org/3/library/archiving.html)

   Python offers several algorithms for this topic.  This tutorial will focus on the most of these methods.  
   
   
   
This project has been uploaded to github (username brianatUCSD): https://github.com/brianatUCSD/Cogs-18-Final-Project

## — How to use zlib: Functions —

This section will explain how to use zlib.  Zlib is a lossless algorithm that can compress as well as decompress files.  

• We start by importing the zlib object

• For the purpose of this project, we also import sys to indicate the size of files to show how sucessful the compression worked.  This will be shown in the function code.


In [ ]:
import zlib

Next we can compress a file of interest using the following.  Parameters are explained in the comments.

In [ ]:
zlib.compress(data, level=-1)

    #data is the data being compressed
    #level is the amount of compression desired, from 0-9

To decompress a file we use: 

In [ ]:
zlib.decompress(data, wbits=MAX_WBITS, bufsize=DEF_BUF_SIZE)

    #data is what is being decompressed
    #wbits controls the window size (max 15)
    #bufsize is used as the initial size of the output buffer (where the buffer is the location in memory where data is temporarily held)



`compressobj` creates a compressed object

In [ ]:
zlib.compressobj(level=-1, method=DEFLATED, wbits=MAX_WBITS, memLevel=DEF_MEM_LEVEL, strategy=Z_DEFAULT_STRATEGY[, zdict])

    # A level of 1 is fastest
    # 'DEFLATED' is the only compression method available
    # wbits controls the window size (max 15)
    # memLevel is the amount of memory used for compression (1-9)
    # Strategy tunes the compression                              
    # zdict is the compression dictionary

`decompressobj` creates a decompresed object

In [ ]:
zlib.decompressobj(wbits=MAX_WBITS[, zdict])


## — How to use gzip: Functions —

Gzip is a type of compression that builds off of the zlib library.

Gzip thus has similar functions like compress and decompress. However for decompress the only input is `data`.

• To return a gzip-compressed file object, use gzip.open

In [ ]:
gzip.open(filename, mode='rb', compresslevel=9, encoding=None, errors=None, newline=None)
    #mode has multiple options for either binary mode or text mode
    #compresslevel takes the same values from 0 - 9
    # encoding, errors, and newline are usually empty for binary mode.

## — How to use zipfile: Functions —

The .ZIP is one of the most common type of compression/archive file formats available.  It differs from the previous implementations in that it can store entire folders easily as opposed to individual text or image files. 

There are many functions used for more advanced applications with data structures, so this with focus on a surface level implication that most users would do.  This will just demonstrate how to create a ZIP file directory.

In [1]:
import zipfile

In [ ]:
file_name = 'sample.zip'
zip_file = ZipFile(file_name, "w")
zip_file.close()

## — How to use gzip: Functions —



# Exploration and Research Application 1: Run Length Encoding

I would now like to explore how to create a very basic algorithm that approaches compression using the definition explained in the introduction.  Run Length Encoding is a type of lossless compression that looks for sequences of repeated characters and then reduces them to avoid redundancy.  

#### First sample code
The first code does not require you to import anything.  It uses recursion to encode input as whatever character followed by the number of iterations of that character.

The following code was found on https://gist.github.com/hltbra/4117933


In [ ]:
def encode(text):
    if not text: #insures input
        return ""
    else:
        last_char = text[0] 
        max_index = len(text)
        i = 1
        while i < max_index and last_char == text[i]:
            i += 1
        return last_char + str(i) + encode(text[i:])

#### Second sample code

This code takes advantage of the itertools library, which consists of functions that help make looping more efficient.  This compression also looks at characters repeated more than 4 times.

This particular instance of itertools uses groupby(), which terminates on the shortest input sequence.  This allows us to differentiate when a new group of characters occurs.

This particular usage was found on a thread on Stackexchange. (https://stackoverflow.com/questions/18948382/run-length-encoding-in-python)


In [ ]:
from itertools import groupby

def runLengthEncode(text):
    result_list = []

    for k,i in groupby(text):  # k and i are variables to loop through the keys and the groups in the input
        run = list(i)
        if(len(run) > 4): #check if it exceeds 4
            result_list.append("/{:02}{}".format(len(run), k))  #append new characters
        else:
            res.extend(run) #extend appends any elements in the list, rather the actual list

    return "".join(result_list)

# Exploration and Research Application 2: Fractal Compression

Fractal compression is especially interesting because it looks for areas of self similarity in an image.  Because of this, it is ideal for natural images (such as landscapes) and combines well with machine learning.  Fractal compression is not ideal for real time applications, as encoding is computationally expensive.  The decoding however is not.  

The following implementation of fractal image compression was made by user pvigier on Github (https://github.com/pvigier/fractal-image-compression). It essentially applies fractal transformations on an image, requiring heavy usage of Python's math library and matplot, a plotting library.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import ndimage
from scipy import optimize
import numpy as np
import math

# Manipulate channels

def get_greyscale_image(img):
    return np.mean(img[:,:,:2], 2)

def extract_rgb(img):
    return img[:,:,0], img[:,:,1], img[:,:,2]

def assemble_rbg(img_r, img_g, img_b):
    shape = (img_r.shape[0], img_r.shape[1], 1)
    return np.concatenate((np.reshape(img_r, shape), np.reshape(img_g, shape), 
        np.reshape(img_b, shape)), axis=2)

# Transformations

def reduce(img, factor):
    result = np.zeros((img.shape[0] // factor, img.shape[1] // factor))
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            result[i,j] = np.mean(img[i*factor:(i+1)*factor,j*factor:(j+1)*factor])
    return result

def rotate(img, angle):
    return ndimage.rotate(img, angle, reshape=False)

def flip(img, direction):
    return img[::direction,:]

def apply_transformation(img, direction, angle, contrast=1.0, brightness=0.0):
    return contrast*rotate(flip(img, direction), angle) + brightness

# Contrast and brightness

def find_contrast_and_brightness1(D, S):
    # Fix the contrast and only fit the brightness
    contrast = 0.75
    brightness = (np.sum(D - contrast*S)) / D.size
    return contrast, brightness 

def find_contrast_and_brightness2(D, S):
    # Fit the contrast and the brightness
    A = np.concatenate((np.ones((S.size, 1)), np.reshape(S, (S.size, 1))), axis=1)
    b = np.reshape(D, (D.size,))
    x, _, _, _ = np.linalg.lstsq(A, b)
    #x = optimize.lsq_linear(A, b, [(-np.inf, -2.0), (np.inf, 2.0)]).x
    return x[1], x[0]

# Compression for greyscale images

def generate_all_transformed_blocks(img, source_size, destination_size, step):
    factor = source_size // destination_size
    transformed_blocks = []
    for k in range((img.shape[0] - source_size) // step + 1):
        for l in range((img.shape[1] - source_size) // step + 1):
            # Extract the source block and reduce it to the shape of a destination block
            S = reduce(img[k*step:k*step+source_size,l*step:l*step+source_size], factor)
            # Generate all possible transformed blocks
            for direction, angle in candidates:
                transformed_blocks.append((k, l, direction, angle, apply_transformation(S, direction, angle)))
    return transformed_blocks

def compress(img, source_size, destination_size, step):
    transformations = []
    transformed_blocks = generate_all_transformed_blocks(img, source_size, destination_size, step)
    for i in range(img.shape[0] // destination_size):
        transformations.append([])
        for j in range(img.shape[1] // destination_size):
            print(i, j)
            transformations[i].append(None)
            min_d = float('inf')
            # Extract the destination block
            D = img[i*destination_size:(i+1)*destination_size,j*destination_size:(j+1)*destination_size]
            # Test all possible transformations and take the best one
            for k, l, direction, angle, S in transformed_blocks:
                contrast, brightness = find_contrast_and_brightness2(D, S)
                S = contrast*S + brightness
                d = np.sum(np.square(D - S))
                if d < min_d:
                    min_d = d
                    transformations[i][j] = (k, l, direction, angle, contrast, brightness)
    return transformations

def decompress(transformations, source_size, destination_size, step, nb_iter=8):
    factor = source_size // destination_size
    height = len(transformations) * destination_size
    width = len(transformations[0]) * destination_size
    iterations = [np.random.randint(0, 256, (height, width))]
    cur_img = np.zeros((height, width))
    for i_iter in range(nb_iter):
        print(i_iter)
        for i in range(len(transformations)):
            for j in range(len(transformations[i])):
                # Apply transform
                k, l, flip, angle, contrast, brightness = transformations[i][j]
                S = reduce(iterations[-1][k*step:k*step+source_size,l*step:l*step+source_size], factor)
                D = apply_transformation(S, flip, angle, contrast, brightness)
                cur_img[i*destination_size:(i+1)*destination_size,j*destination_size:(j+1)*destination_size] = D
        iterations.append(cur_img)
        cur_img = np.zeros((height, width))
    return iterations

# Compression for color images

def reduce_rgb(img, factor):
    img_r, img_g, img_b = extract_rgb(img)
    img_r = reduce(img_r, factor)
    img_g = reduce(img_g, factor)
    img_b = reduce(img_b, factor)
    return assemble_rbg(img_r, img_g, img_b)

def compress_rgb(img, source_size, destination_size, step):
    img_r, img_g, img_b = extract_rgb(img)
    return [compress(img_r, source_size, destination_size, step), \
        compress(img_g, source_size, destination_size, step), \
        compress(img_b, source_size, destination_size, step)]

def decompress_rgb(transformations, source_size, destination_size, step, nb_iter=8):
    img_r = decompress(transformations[0], source_size, destination_size, step, nb_iter)[-1]
    img_g = decompress(transformations[1], source_size, destination_size, step, nb_iter)[-1]
    img_b = decompress(transformations[2], source_size, destination_size, step, nb_iter)[-1]
    return assemble_rbg(img_r, img_g, img_b)

# Plot

def plot_iterations(iterations, target=None):
    # Configure plot
    plt.figure()
    nb_row = math.ceil(np.sqrt(len(iterations)))
    nb_cols = nb_row
    # Plot
    for i, img in enumerate(iterations):
        plt.subplot(nb_row, nb_cols, i+1)
        plt.imshow(img, cmap='gray', vmin=0, vmax=255, interpolation='none')
        if target is None:
            plt.title(str(i))
        else:
            # Display the RMSE
            plt.title(str(i) + ' (' + '{0:.2f}'.format(np.sqrt(np.mean(np.square(target - img)))) + ')')
        frame = plt.gca()
        frame.axes.get_xaxis().set_visible(False)
        frame.axes.get_yaxis().set_visible(False)
    plt.tight_layout()

# Parameters

directions = [1, -1]
angles = [0, 90, 180, 270]
candidates = list(zip(directions, angles))

# Tests

def test_greyscale():
    img = mpimg.imread('monkey.gif')
    img = get_greyscale_image(img)
    img = reduce(img, 4)
    plt.figure()
    plt.imshow(img, cmap='gray', interpolation='none')
    transformations = compress(img, 8, 4, 8)
    iterations = decompress(transformations, 8, 4, 8)
    plot_iterations(iterations, img)
    plt.show()

def test_rgb():
    img = mpimg.imread('lena.gif')
    img = reduce_rgb(img, 8)
    transformations = compress_rgb(img, 8, 4, 8)
    retrieved_img = decompress_rgb(transformations, 8, 4, 8)
    plt.figure()
    plt.subplot(121)
    plt.imshow(np.array(img).astype(np.uint8), interpolation='none')
    plt.subplot(122)
    plt.imshow(retrieved_img.astype(np.uint8), interpolation='none')
    plt.show()
                    
if __name__ == '__main__':
    test_greyscale()
    #test_rgb()

# Project Functions

`functions.py` in the my_module folder will compress a file.  The program will ask for a file name as input, followed by the level of desired compression.  It will then print out the compressed version of the file.  A sample file is given in the directory, called 'notes.txt'.

It will also have functions that use the exploratory run length encoding algorithm.

The functions have onto them added a size comparison output to show the efficiency of the compression.

The scripts work, but must be used in the same directory as the functions.